In [ ]:
import pandas as pd

df = pd.read_csv('netlogbase.csv')

# drop unnecessary features
df = df.drop(columns=['timestamp'])
df = df.drop(columns=['src_ip'])
df = df.drop(columns=['dst_ip'])
df = df.drop(columns=['protocol_name'])

<ipython-input-56-06511105d767>:3: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('netlogbase.csv')


In [ ]:
df

,protocol,length,src_port,dst_port,flags,ttl,payload_size
0,17,305.0,54915.0,54915.0,NaN,128.0,263.0
1,6,82.0,50315.0,443.0,PA,128.0,28.0
2,6,54.0,443.0,50315.0,A,55.0,0.0
3,6,78.0,443.0,50315.0,PA,55.0,24.0
4,17,666.0,55001.0,3702.0,NaN,1.0,624.0
...,...,...,...,...,...,...,...
16341567,92,137.0,137.0,NaN,64,50.0,UDP
16341568,92,137.0,137.0,NaN,64,50.0,UDP
16341569,92,137.0,137.0,NaN,64,50.0,UDP
16341570,92,137.0,137.0,NaN,64,50.0,UDP


In [ ]:
# Use a hashmap (dictionary in Python) to store seen rows
seen_rows = {}
unique_rows = []

for index, row in df.iterrows():
    row_tuple = tuple(row)  # Convert row to a tuple (hashable)
    if row_tuple not in seen_rows:
        seen_rows[row_tuple] = True
        unique_rows.append(row)

# Create a new DataFrame with unique rows
unique_df = pd.DataFrame(unique_rows, columns=df.columns)

unique_df

,protocol,length,src_port,dst_port,flags,ttl,payload_size
0,17,305.0,54915.0,54915.0,NaN,128.0,263.0
1,6,82.0,50315.0,443.0,PA,128.0,28.0
2,6,54.0,443.0,50315.0,A,55.0,0.0
3,6,78.0,443.0,50315.0,PA,55.0,24.0
4,17,666.0,55001.0,3702.0,NaN,1.0,624.0
...,...,...,...,...,...,...,...
16341346,64,59771.0,5355.0,NaN,1,22.0,UDP
16341364,171,443.0,53925.0,PA,56,113.0,TCP
16341522,82,58246.0,1947.0,NaN,128,40.0,UDP
16341548,75,64930.0,5355.0,NaN,1,33.0,UDP


In [ ]:
# resetting index
unique_df = unique_df.reset_index(drop=True)

In [ ]:
# write to csv
unique_df.to_csv('unique_df_new.csv', index=False)

In [ ]:
import pandas as pd
unique_df = pd.read_csv('unique_df_new.csv')

<ipython-input-175-7571424c9fcb>:2: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  unique_df = pd.read_csv('unique_df_new.csv')


In [ ]:
unique_df.head()

,protocol,length,src_port,dst_port,flags,ttl,payload_size
0,17,305.0,54915.0,54915.0,NaN,128.0,263.0
1,6,82.0,50315.0,443.0,PA,128.0,28.0
2,6,54.0,443.0,50315.0,A,55.0,0.0
3,6,78.0,443.0,50315.0,PA,55.0,24.0
4,17,666.0,55001.0,3702.0,NaN,1.0,624.0


In [ ]:
import pandas as pd
import numpy as np

unique_df = pd.read_csv('unique_df_new.csv')

unique_df = unique_df.fillna(0)

unique_df['flags'] = unique_df['flags'].map({'A': 1, 'PA': 2,
                                             'FA': 3, 'S': 4, 'SA': 5,
                                             'RA': 6, 'R': 7, 'FPA': 8})
unique_df = unique_df.fillna(0)

# drop all rows with NaN in src_port, dst_port, and payload_size
# unique_df = unique_df.dropna(subset=['payload_size', 'dst_port'])

# feature-label split
X = unique_df.values  # feature set


<ipython-input-201-338f1640cdd5>:4: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  unique_df = pd.read_csv('unique_df_new.csv')


In [ ]:
unique_df

,protocol,length,src_port,dst_port,flags,ttl,payload_size
0,17,305.0,54915.0,54915.0,0.0,128.0,263.0
1,6,82.0,50315.0,443.0,2.0,128.0,28.0
2,6,54.0,443.0,50315.0,1.0,55.0,0.0
3,6,78.0,443.0,50315.0,2.0,55.0,24.0
4,17,666.0,55001.0,3702.0,0.0,1.0,624.0
...,...,...,...,...,...,...,...
609390,64,59771.0,5355.0,0,0.0,22.0,UDP
609391,171,443.0,53925.0,PA,0.0,113.0,TCP
609392,82,58246.0,1947.0,0,0.0,40.0,UDP
609393,75,64930.0,5355.0,0,0.0,33.0,UDP


In [ ]:
# remove rows in dst_port and payload_size that cannot be converted to int
unique_df = unique_df[pd.to_numeric(unique_df['dst_port'], errors='coerce').notna()]
unique_df = unique_df[pd.to_numeric(unique_df['payload_size'], errors='coerce').notna()]
# remove all strings from dst_port
unique_df['dst_port'] = unique_df['dst_port'].str.replace(r'\D', '', regex=True)
# remove all strings from payload_size
unique_df['payload_size'] = unique_df['payload_size'].str.replace(r'\D', '', regex=True)
# remove all NaN values from dst_port and payload_size
unique_df = unique_df.dropna(subset=['dst_port', 'payload_size'])
# convert to int
unique_df['dst_port'] = unique_df['dst_port'].astype(int)
unique_df['payload_size'] = unique_df['payload_size'].astype(int)

In [ ]:
unique_df

,protocol,length,src_port,dst_port,flags,ttl,payload_size
524288,17,1292.0,443.0,550470,0.0,55.0,12500
524289,17,209.0,55047.0,4430,0.0,128.0,1670
524290,17,560.0,443.0,550470,0.0,55.0,5180
524291,17,163.0,443.0,550470,0.0,55.0,1210
524292,17,73.0,55047.0,4430,0.0,128.0,310
...,...,...,...,...,...,...,...
608142,6,341.0,443.0,51580,2.0,51.0,287
608143,6,115.0,443.0,51580,2.0,51.0,61
608144,6,85.0,443.0,51580,2.0,51.0,31
608145,6,85.0,51580.0,443,2.0,128.0,31


In [ ]:
unique_df = unique_df.reset_index(drop=True)
unique_df.to_csv('unique_df_clean.csv', index=False)

In [ ]:
unique_df.columns

Index(['protocol', 'length', 'src_port', 'dst_port', 'flags', 'ttl',
       'payload_size'],
      dtype='object')

In [ ]:
for col in unique_df.columns:
    if unique_df[col].dtype != 'int':
        try:
            unique_df[col] = unique_df[col].astype(int)
        except ValueError:
            print(f"Column '{col}' cannot be converted to int")

# remove rows which cannot be converted to int



In [ ]:
unique_df

,protocol,length,src_port,dst_port,flags,ttl,payload_size
0,17,1292,443,550470,0,55,12500
1,17,209,55047,4430,0,128,1670
2,17,560,443,550470,0,55,5180
3,17,163,443,550470,0,55,1210
4,17,73,55047,4430,0,128,310
...,...,...,...,...,...,...,...
77714,6,341,443,51580,2,51,287
77715,6,115,443,51580,2,51,61
77716,6,85,443,51580,2,51,31
77717,6,85,51580,443,2,128,31


In [ ]:
#has_nan = df.isna().any().any()
#print(has_nan)
nan_in_columns = unique_df.isna().any()

print(nan_in_columns)

protocol        False
length          False
src_port        False
dst_port        False
flags           False
ttl             False
payload_size    False
dtype: bool


In [ ]:
for col in unique_df.columns:
    if not pd.api.types.is_integer_dtype(unique_df[col]):
        print(f"Column '{col}' does not contain only integers.")
# no output means theyre all ints

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

# Predict on the full dataset
isolation_forest = IsolationForest(contamination= .025, bootstrap = True, verbose = 1)
unique_df_normalized = pd.DataFrame(scaler.fit_transform(unique_df), columns=unique_df.columns)
isolation_forest.fit(unique_df_normalized)
full_output = isolation_forest.predict(unique_df_normalized)

unique, counts = np.unique(full_output, return_counts=True)
results = dict(zip(unique, counts))
print("Prediction results:", results)

anomalies_full = unique_df_normalized[full_output == -1]
normals_full = unique_df_normalized[full_output == 1]

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.2s


Prediction results: {np.int64(-1): np.int64(1936), np.int64(1): np.int64(75783)}


In [ ]:
anomalies_full

,protocol,length,src_port,dst_port,flags,ttl,payload_size
200,0.0,8.235127,1.022682,-0.816594,0.5,0.178082,8.194915
202,0.0,6.895184,1.024051,-0.816594,0.5,0.178082,6.858757
346,1.0,3.371105,0.000000,0.256967,-0.5,-0.821918,3.378531
600,0.0,14.317280,1.022682,-0.816594,0.5,0.178082,14.259887
739,0.0,6.164306,1.022682,-0.816594,0.5,0.178082,6.129944
...,...,...,...,...,...,...,...
77666,0.0,3.855524,0.000000,-0.738619,0.5,-0.876712,0.273446
77670,0.0,4.031161,0.000000,-0.738619,0.5,-0.876712,0.290960
77693,1.0,3.413598,0.000000,-0.736466,-0.5,-0.808219,0.231638
77707,0.0,-0.070822,0.000000,-0.738618,2.0,-0.876712,-0.121469


In [ ]:
# A new packet
X_anomalous = scaler.transform(np.array([[-1.54545455, -0.05172414, -0.93623388, -0.79454144,  0.,          0.,
 -0.17613636]] ))

print(X_anomalous)

prediction = isolation_forest.predict(X_anomalous)  # [-1] or [1]
score = isolation_forest.decision_function(X_anomalous)

print(prediction[0])

print("New packet is:", "Anomaly" if prediction[0] == -1 else "Normal")
print("Novelty score:", score[0])  # More positive = more normal

[[-0.68595041 -0.2579369  -0.00768954 -0.8239218  -0.5        -1.57534247
  -0.12151868]]
1
New packet is: Normal
Novelty score: 0.02625695018786478


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


In [ ]:
import joblib

# Save
joblib.dump(scaler, 'scalerTry5.joblib')
joblib.dump(isolation_forest, 'iforestTry5.joblib')


['iforestTry5.joblib']